# Dataset creation to train a YoloV8 model for tank detection

This notebook shows how to build a dataset of annotated images to train a computer vision model for object detection. We use available open source datasets to create a dataset of military vehicles and format it correctly for YoloV8 training.

We use [fiftyone](https://github.com/voxel51/fiftyone) to convert, merge, label and format the images prior to training with [Yolov8](https://github.com/ultralytics/ultralytics).

### Setup

We start by setting up some logging.

In [1]:
import logging

logging.basicConfig(level=logging.INFO)

### Download images from ImageNet

The first dataset we'll use is ImageNet21k. The ImageNet21k dataset is available at [https://image-net.org/download-images.php](https://image-net.org/download-images.php). You need to register and be granted access to download the images. We use the Winter 21 version since it gives the option of downloading the images for a single synset: https://image-net.org/data/winter21_whole/SYNSET_ID.tar, e.g., https://image-net.org/data/winter21_whole/n02352591.tar. The processed version of ImageNet21k is available here : https://github.com/Alibaba-MIIL/ImageNet21K. The class ids and names are available here https://github.com/google-research/big_transfer/issues/7#issuecomment-640048775.

We'll begin by downloading the class names that are in ImageNet21k and look for relevant classes that we can use.

In [2]:
from pathlib import Path

imagenet_dir = Path() / "imagenet"

In [3]:
from adomvi.datasets.imagenet import download_class_names, find_class_by_text

classes = download_class_names(imagenet_dir)
find_class_by_text(classes, "military")

INFO:root:File imagenet/imagenet21k_wordnet_ids.txt already exists. Skipping download.
INFO:root:File imagenet/imagenet21k_wordnet_lemmas.txt already exists. Skipping download.


{'n03762982': 'military_hospital',
 'n03763727': 'military_quarters',
 'n03763968': 'military_uniform',
 'n03764276': 'military_vehicle',
 'n04552348': 'warplane, military_plane',
 'n08249459': 'concert_band, military_band',
 'n09809538': 'army_engineer, military_engineer',
 'n09943239': 'commissioned_military_officer',
 'n10316360': 'military_attache',
 'n10316527': 'military_chaplain, padre, Holy_Joe, sky_pilot',
 'n10316862': 'military_leader',
 'n10317007': 'military_officer, officer',
 'n10317500': 'military_policeman, MP',
 'n10512372': 'recruit, military_recruit',
 'n10582746': 'serviceman, military_man, man, military_personnel',
 'n10759331': 'volunteer, military_volunteer, voluntary'}

We can now download images and annotations for the relevant classes. The `download_imagenet_detections` function will download the images and annotations for the given class ids **if the annotations exist** (not all classes have been annotated).

In [4]:
from adomvi.datasets.imagenet import download_imagenet_detections

class_ids = ["n02740300", "n04389033", "n02740533", "n04464852", "n03764276"]
download_imagenet_detections(class_ids, imagenet_dir)

INFO:root:File imagenet/bboxes_annotations.tar.gz already exists. Skipping download.
INFO:root:There are not annotations for class n02740300.
INFO:root:Annotations directory imagenet/labels/n04389033 already exists. Skipping extract.
INFO:root:There are not annotations for class n02740533.
INFO:root:There are not annotations for class n04464852.
INFO:root:There are not annotations for class n03764276.
INFO:root:Deleting annotations dir.


The data we just downloaded into the `imagenet` directory is not all clean: there are annotations which have no corresponding image. We need to remove those labels, otherwise this causes errors when importing the data into fiftyone.

In [5]:
from adomvi.datasets.imagenet import cleanup_labels_without_images

cleanup_labels_without_images(imagenet_dir)

INFO:root:Deleting 0 labels without images


We can now create a new dataset with `fiftyone`. Fiftyone allows us to manage images annotated with bounding boxes and labels, to merge datasets from different sources, and to split the datasets and prepare them for processing.

In [6]:
from adomvi.utils import cleanup_existing_dataset

imagenet_name = "military-vehicles"
cleanup_existing_dataset(imagenet_name)

INFO:root:Dataset 'military-vehicles' deleted.


In [7]:
import fiftyone as fo

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=imagenet_dir,
    dataset_type=fo.types.VOCDetectionDataset,
    # dataset_name = imagenet_name,
)

dataset.name = imagenet_name

dataset.map_labels(
    "ground_truth",
    {"n04389033":"AFV"}
).save()

 100% |█████████████████| 378/378 [470.4ms elapsed, 0s remaining, 806.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 378/378 [470.4ms elapsed, 0s remaining, 806.4 samples/s]      


Once our dataset is created, we can launch a session to display the dataset and view the annotated images

In [8]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [9]:
session.show()

### Add OpenImage samples

The ImageNet dataset only contained 378 annotated images of tanks, so we'll look into other available datasets to improve training of the model. We’ll load [Open Images](https://storage.googleapis.com/openimages/web/index.html) samples with `Tank` detection labels, passing in `only_matching=True` to only load the `Tank` labels. We then map these labels by changing `Tank` into `tank`.

In [10]:
import fiftyone.zoo as foz

oi_samples = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Tank"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Tank":"AFV"}
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/home/ukemkata/fiftyone/open-images-v7/train' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'train' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'train' is sufficient


...

INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'open-images-v7'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'open-images-v7'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


We can add these new samples into our training dataset with `merge_samples()`:

In [11]:
dataset.merge_samples(oi_samples)

In [12]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [13]:
session.show()

### Add Roboflow dataset

The current data contained 1624 annotated images of tank, so we'll look into other available datasets to improve training of the model. We’ll load [Roboflow Images](https://universe.roboflow.com/) with `Tank` detection labels.

In [14]:
from adomvi.datasets.roboflow import download_roboflow_dataset

roboflow_dir = Path() / "roboflow"
url="https://universe.roboflow.com/ds/P2jPq32qKU?key=E4MIo8mavP"
download_roboflow_dataset(url, roboflow_dir)

INFO:root:Downloading roboflow/dataset_rf.zip from https://universe.roboflow.com/ds/P2jPq32qKU?key=E4MIo8mavP ...
INFO:root:Download complete.
INFO:root:Extracted to roboflow


In [15]:
from adomvi.datasets.roboflow import restructure_dataset

restructure_dataset(roboflow_dir)

INFO:root:Dataset dir restructured successfully.


In [16]:
roboflow_name = "russian-military-annotated"
cleanup_existing_dataset(roboflow_name)

INFO:root:Dataset 'russian-military-annotated' does not exist.


In [17]:
# Import the roboflow dataset
dataset_rf = fo.Dataset.from_dir(
    dataset_dir=roboflow_dir,
    dataset_type=fo.types.VOCDetectionDataset,
    name = roboflow_name,
)

# Define the existing labels and their mappings
label_mapping = {
    "bm-21": "AFV",
    "t-80": "AFV",
    "t-64": "AFV",
    "t-72": "AFV",
    "bmp-1": "AFV",
    "bmp-2": "AFV",
    "bmd-2": "AFV",
    "btr-70": "APC",
    "btr-80": "APC",
    "mt-lb": "APC",
}

# Map the labels
dataset_rf.map_labels(
    "ground_truth",
    label_mapping
).save()

 100% |███████████████| 1042/1042 [1.5s elapsed, 0s remaining, 700.0 samples/s]         


INFO:eta.core.utils: 100% |███████████████| 1042/1042 [1.5s elapsed, 0s remaining, 700.0 samples/s]         


In [18]:
# from adomvi.datasets.roboflow import delete_images_without_labels

# delete_images_without_labels(dataset_rf)

We can add these new samples into our training dataset

In [19]:
dataset.merge_samples(dataset_rf)

In [20]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [21]:
session.show()

### Add google dataset

We provide a sample annotated dataset with 4 classes (*AFV*, *APC*, *LAV* & *MEV*). You can download the dataset from [here](https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz) and extract it.

In [22]:
from adomvi.datasets.google import download_google_dataset

google_dir = Path() / "google"
url = "https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz"

download_google_dataset(url, google_dir)

INFO:root:Downloading google/military-vehicles-dataset.tar.gz from https://github.com/jonasrenault/adomvi/releases/download/v1.2.0/military-vehicles-dataset.tar.gz ...
INFO:root:Download complete.
INFO:root:Extracted to google


We'll use fiftyone to load and preview the dataset.

In [23]:
google_name = "google-military-vehicles"
cleanup_existing_dataset(google_name)

INFO:root:Dataset 'google-military-vehicles' deleted.


In [24]:
dataset_google_dir = google_dir / "dataset"

# Create the dataset
dataset_google = fo.Dataset.from_dir(
    dataset_dir=dataset_google_dir,
    dataset_type=fo.types.YOLOv4Dataset,
    name=google_name,
)

Images file '/home/ukemkata/workspace/adomvi2/notebooks/google/dataset/images.txt' not found. Listing data directory '/home/ukemkata/workspace/adomvi2/notebooks/google/dataset/data/' instead


 100% |█████████████████| 669/669 [599.6ms elapsed, 0s remaining, 1.1K samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 669/669 [599.6ms elapsed, 0s remaining, 1.1K samples/s]      


We can add these new samples into our training dataset

In [25]:
dataset.merge_samples(dataset_google)


In [26]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.


In [27]:
session.show()

### Export dataset to disk

Now that our dataset is created, we'll export it into a format supported by YOLOv8 to train our model.

We first remove tags from the dataset, and split it into a train, val and test sets.

In [28]:
import fiftyone.utils.random as four

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(dataset, {"train": 0.8, "val": 0.1, "test": 0.1})

Once our dataset is split, we can export it to a specific directory.

In [29]:
from adomvi.yolo.utils import export_yolo_data

export_dir = Path() / "dataset"
export_yolo_data(dataset, export_dir, ["AFV", "APC", "MEV", "LAV"], split = ["train", "val", "test"], overwrite=True)

 100% |███████████████| 2668/2668 [2.5s elapsed, 0s remaining, 1.3K samples/s]       


INFO:eta.core.utils: 100% |███████████████| 2668/2668 [2.5s elapsed, 0s remaining, 1.3K samples/s]       


Directory 'dataset' already exists; export will be merged with existing files


 100% |█████████████████| 334/334 [345.3ms elapsed, 0s remaining, 967.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 334/334 [345.3ms elapsed, 0s remaining, 967.2 samples/s]      


Directory 'dataset' already exists; export will be merged with existing files


 100% |█████████████████| 333/333 [314.3ms elapsed, 0s remaining, 1.1K samples/s]       


INFO:eta.core.utils: 100% |█████████████████| 333/333 [314.3ms elapsed, 0s remaining, 1.1K samples/s]       
